# NMF Fitting

In [26]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [27]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import gensim
import time

from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):  # loop through each row of H.  idx = row index.  topic = actual row
        print("\nTopic %d:" % (idx))
        #print([(vectorizer.get_feature_names()[i], topic[i])  # printing out words corresponding to indices found in next line
                        #for i in topic.argsort()[:-top_n - 1:-1]])  # finding indices of top words in topic
            
        print_list = [(vectorizer.get_feature_names()[i], topic[i])  
                        for i in topic.argsort()[:-top_n - 1:-1]]
        for item in print_list:
            print(item)

In [29]:
#TODO:Input a dataframe that are ai related abstracts, need variables: final_frqwds_removed
abstracts = pd.read_csv(r'/home/zz3hs/git/dspg21RnD/data/dspg21RnD/bert_ai_abstracts.csv') 
abstracts2 = pd.read_csv(r'/home/zz3hs/git/dspg21RnD/data/dspg21RnD/bert_ai_abstracts_2.csv')   

In [30]:
lim_docs = abstracts["final_frqwds_removed"]
len(lim_docs)

lim_docs2 = abstracts2["final_frqwds_removed"]
len(lim_docs2)

7658

In [31]:
# input needed for LDA, NMF (all from Scikit-Learn) is one string per document (not a list of strings)

text1 = []

for token_list in lim_docs:
    text1.append(token_list)
    
    
    
text2 = []

for token_list in lim_docs2:
    text2.append(token_list)

In [32]:
# Create a TF-IDF document-term matrix for the AI corpus 

# TRY DIFFERENT PARAMETERS IN THE TF-IDF DOC-TERM MATRIX SET-UP
nmf_vectorizer1 = TfidfVectorizer(max_df=1.0, min_df=3, lowercase=True) #, max_features=int(len(lim_docs)/2))
nmf_vectorizer2 = TfidfVectorizer(max_df=1.0, min_df=3, lowercase=True) #, max_features=int(len(lim_docs)/2))

# by default TfidfVectorizer has l2 normalization for rows: 
# from Scikit Learn documentation: Each output row will have unit norm, either: * ‘l2’: Sum of squares of vector 
# elements is 1. The cosine similarity between two vectors is their dot product when l2 norm has been applied.

nmf_tf_idf1 = nmf_vectorizer1.fit_transform(text1)
nmf_tf_idf1.shape


nmf_tf_idf2 = nmf_vectorizer2.fit_transform(text2)
nmf_tf_idf2.shape

(7658, 10966)

# Mean+2 SD (k=30)

In [36]:
# topic modeling with NMF
nmf_model1 = NMF(n_components=30, random_state=1)  # TRY DIFFERENT NUMBERS OF TOPICS
W1 = nmf_model1.fit_transform(nmf_tf_idf1)
H1 = nmf_model1.components_

/home/zz3hs/.conda/envs/crystal_bert/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


In [42]:
print_topics(nmf_model1, nmf_vectorizer1, 15)


Topic 0:
('science', 6.616599589937207)
('scientific', 3.1381582772920447)
('scientist', 1.9075292488139122)
('computer', 1.2368303055758423)
('collaboration', 1.029675148397276)
('interdisciplinary', 0.6380691243414814)
('discipline', 0.623774406490787)
('cyberinfrastructure', 0.6074970571388717)
('citizen', 0.5192391939913021)
('discovery', 0.46285138122418207)
('inquiry', 0.4483034097959139)
('career', 0.43773341539894634)
('woman', 0.4289214863281009)
('national', 0.4246896161448243)
('public', 0.4127100435870789)

Topic 1:
('brain', 4.088675361595652)
('neuroscience', 0.7736573455926538)
('imaging', 0.7357418427089162)
('functional', 0.7080715164100668)
('fmri', 0.5613019289566987)
('region', 0.5538502161500122)
('connectivity', 0.5322248778401593)
('neuroimaging', 0.45215808460245205)
('mri', 0.35063392469887916)
('disorder', 0.3161360536413738)
('resolution', 0.20822802245248415)
('magnetic_resonance', 0.20049245379695935)
('structural', 0.19200021129412184)
('activation', 0.18

# Mean+2.5 SD(k=30)

In [40]:
#topic modeling with NMF

nmf_model2 = NMF(n_components=30, random_state=1)  # TRY DIFFERENT NUMBERS OF TOPICS
W2 = nmf_model2.fit_transform(nmf_tf_idf2)
H2 = nmf_model2.components_
#print_topics(nmf_model, nmf_vectorizer, 10)

In [41]:
print_topics(nmf_model2, nmf_vectorizer2, 15)


Topic 0:
('algorithm', 6.249506598539285)
('machine_learning', 1.2051828863356444)
('algorithmic', 0.9797292672365375)
('complexity', 0.9470641328327748)
('efficient', 0.8701540818257613)
('graph', 0.8505736784975366)
('theory', 0.7829503887503898)
('approximation', 0.7141586468066781)
('pi', 0.6829233850299871)
('bound', 0.5422695796339557)
('search', 0.5410185893850856)
('theoretical', 0.5104793818208051)
('fundamental', 0.4888592267487909)
('polynomial', 0.4883295433694804)
('linear', 0.4638699771395042)

Topic 1:
('neural', 2.215583557480533)
('circuit', 0.9325391370532041)
('neuron', 0.9020069617251519)
('sensory', 0.3956928347773224)
('cortical', 0.3468230436954055)
('animal', 0.3391700771281825)
('dynamics', 0.31628712529699815)
('neuronal', 0.26236375375052373)
('behavioral', 0.25561149188848303)
('motor', 0.25144141692696936)
('stimuli', 0.24059633785403756)
('disorder', 0.21808003296322395)
('cell', 0.2113898995577242)
('population', 0.20992065226297207)
('action', 0.1863074